# item_cf

In [78]:
!python baseline0_itemCF.py

gen train_data_matrix begin...
cal cosine_similarity begin...
cal cosine_similarity end
100%|████████████████████████████████████| 27195/27195 [00:45<00:00, 600.60it/s]
935.4970018863678s
complete!!!


# A simple itemCF Baseline, score:0.0767

In [6]:
!python baseline1_itemCF.py

phase: 0
phase: 1
100%|██████████████████████████████████████| 3389/3389 [00:12<00:00, 262.74it/s]
27.01650309562683s
complete!!!


In [1]:
#!/usr/bin/env python  
# -*- coding:utf-8 -*-  
  
import pandas as pd  
from tqdm import tqdm  
from collections import defaultdict  
import math  
  
def get_sim_item(df, user_col, item_col, use_iif=False):  
    user_item_ = df.groupby(user_col)[item_col].agg(set).reset_index()  
    user_item_dict = dict(zip(user_item_[user_col], user_item_[item_col]))  
  
    sim_item = {}  # sim_item[i][j] 表示item_i和item_j共现的次数
    item_cnt = defaultdict(int)  # 统计某个item有多少个user购买过
    for user, items in tqdm(user_item_dict.items()):  
        for i in items:  
            item_cnt[i] += 1  
            sim_item.setdefault(i, {})  
            for relate_item in items:  
                if i == relate_item:  
                    continue  
                sim_item[i].setdefault(relate_item, 0)  
                if not use_iif:  
                    sim_item[i][relate_item] += 1  
                else:  
                    sim_item[i][relate_item] += 1 / math.log(1 + len(items))  
    
    sim_item_corr = sim_item.copy()  
    for i, related_items in tqdm(sim_item.items()):  
        for j, cij in related_items.items():  
            # item_i和item_j共现的次数/各自出现的次数之积
            sim_item_corr[i][j] = cij/math.sqrt(item_cnt[i]*item_cnt[j])  
  
    return sim_item_corr, user_item_dict  
  

def recommend(sim_item_corr, user_item_dict, user_id, top_k, item_num):  
    """
    向user_id推荐item_num个该user_id没有买过的item
    """
    rank = {}  
    # 该user_id购买过的items
    interacted_items = user_item_dict[user_id]  
    # 遍历该user购买过的items，
    for i in interacted_items:  
        # 遍历该item共现最高的top_k个item，把其中用户没有买过的加入推荐列表
#         for j, wij in sorted(sim_item_corr[i].items(), reverse=True)[:top_k]:  
        for j, wij in sorted(sim_item_corr[i].items(), key=lambda d: d[1], reverse=True)[:top_k]:  
            if j not in interacted_items:  
                rank.setdefault(j, 0)  
                rank[j] += wij  
    return sorted(rank.items(), key=lambda d: d[1], reverse=True)[:item_num]  
  
def get_predict(df, pred_col, top_fill):  
    """
    fill user to 50 items
    逻辑就是如果推荐给用户的items少于50个，就用点击数最高的item补足
    """
    top_fill = [int(t) for t in top_fill.split(',')]  
    scores = [-1 * i for i in range(1, len(top_fill) + 1)]  
    ids = list(df['user_id'].unique())  
    fill_df = pd.DataFrame(ids * len(top_fill), columns=['user_id'])  
    fill_df.sort_values('user_id', inplace=True)  
    fill_df['item_id'] = top_fill * len(ids)  
    fill_df[pred_col] = scores * len(ids)  
    df = df.append(fill_df)  
    df.sort_values(pred_col, ascending=False, inplace=True)  
    df = df.drop_duplicates(subset=['user_id', 'item_id'], keep='first')  
    df['rank'] = df.groupby('user_id')[pred_col].rank(method='first', ascending=False)  
    df = df[df['rank'] <= 50]  
    df = df.groupby('user_id')['item_id'].apply(lambda x: ','.join([str(i) for i in x])).str.split(',', expand=True).reset_index()  
    return df  

In [2]:
now_phase = 1  
train_path = './data_origin/underexpose_train'  
test_path = './data_origin/underexpose_test'  
recom_item = []  

whole_click = pd.DataFrame()  
click_test = pd.DataFrame()  
for c in range(now_phase + 1):  
    print('phase:', c)  
    click_train = pd.read_csv(train_path + '/underexpose_train_click-{}.csv'.format(c), header=None,  names=['user_id', 'item_id', 'time'])  
    click_test1 = pd.read_csv(test_path + '/underexpose_test_click-{}/underexpose_test_click-{}.csv'.format(c, c), header=None,  names=['user_id', 'item_id', 'time'])  

    all_click = click_train.append(click_test1)  
    click_test = click_test.append(click_test1) 
    whole_click = whole_click.append(all_click)  
    
item_sim_list, user_item = get_sim_item(whole_click, 'user_id', 'item_id', use_iif=True)  

for i in tqdm(click_test['user_id'].unique()):  
    rank_item = recommend(item_sim_list, user_item, i, 500, 50)  
    for j in rank_item:  
        recom_item.append([i, j[0], j[1]])  
# find most popular items  
top50_click = whole_click['item_id'].value_counts().index[:50].values  
top50_click = ','.join([str(i) for i in top50_click])  

recom_df = pd.DataFrame(recom_item, columns=['user_id', 'item_id', 'sim'])  
result = get_predict(recom_df, 'sim', top50_click)  
result.to_csv('baseline.csv', index=False, header=None)

phase: 0
phase: 1


100%|██████████| 3389/3389 [00:12<00:00, 264.52it/s]
